In [1]:
import os
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

In [ ]:
def thresh_split_chanel(img, thresh1, thresh2):
    gray1 = img[:,:,1]
    mask1 = cv.inRange(gray1, 0, thresh1) #90
    gray2 = img[:,:,2]
    mask2 = cv.inRange(gray2, thresh2, 255) #130
    # combine white part of both images
    mask = cv.bitwise_or(mask1, mask2)
    return mask

In [2]:
def center_crop(img, cut_width=0, cut_height=0):
    width, height = img.shape[1], img.shape[0]
    new_width = width - cut_width
    new_height = height - cut_height
    left = (width - new_width) / 2
    top = (height - new_height) / 2
    right = (width + new_width) / 2
    bottom = (height + new_height) / 2
    return img[int(top):int(bottom), int(left):int(right)]

In [3]:
def crop_rotation(image, box, height, width):
    

    src_pts = box.astype("float32")
    dst_pts = np.array([[0, height],
                        [0, 0],
                        [width, 0],
                        [width, height]], dtype="float32")

    M = cv.getPerspectiveTransform(src_pts, dst_pts)

    # directly warp the rotated rectangle to get the straightened rectangle
    warped = cv.warpPerspective(image, M, (width, height))
    return warped

In [4]:
import os
len(os.listdir('./'))

5

In [5]:
def segmentation(img_path, background=False, rotation=False):

    global count
    assert os.path.exists(img_path), "File does not exist"
    if background == False and rotation == False:
        output_dir = "/home/dotronghiep/Documents/Datasets/Peanuts/test/"
    elif background == False and rotation == True:
        output_dir = "/home/dotronghiep/Documents/Datasets/Peanuts/no_background_rotation/"
    elif background == True and rotation == False:
        output_dir = "/home/dotronghiep/Documents/Datasets/Peanuts/background_no_rotation/"
    else:
        output_dir = "/home/dotronghiep/Documents/Datasets/Peanuts/background_rotation/"

    min_height = 30
    min_width = 30

    extend_box = 10
    # Đọc ảnh và chuyển đổi sang grayscale
    img = cv.imread(img_path)
    img = center_crop(img, 300)
    gray = img[:,:,2]

    threshold_value = 140
    # Áp dụng ngưỡng
    _, thresh = cv.threshold(gray, threshold_value, 255, cv.THRESH_BINARY_INV)

    thresh = cv.bitwise_not(thresh)

    # Loại bỏ nhiễu và xác định vùng nền chắc chắn
    kernel = np.ones((3, 3), np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=1)
    sure_bg = cv.dilate(opening, kernel, iterations=1)

    # Xác định vùng chữ chắc chắn
    dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
    _, sure_fg = cv.threshold(dist_transform, 0.001 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)

    # Xác định vùng không xác định
    unknown = cv.subtract(sure_bg, sure_fg)

    # Nhãn các vùng
    _, markers = cv.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Watershed
    markers = cv.watershed(img, markers)

    # Tăng kích thước các đường viền
    markers = markers.astype(np.uint8)
    kernel_dilate = np.ones((3, 3), np.uint8)
    markers = cv.dilate(markers, kernel_dilate, iterations=1)

    non_background = img.copy()
    non_background[markers==1] = 0

    objects = np.logical_and(markers != 1, markers != 255)

    # Find contours in the binary_markers image
    contours, _ = cv.findContours(objects.astype(np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if background==False:
        base_image = non_background
    else:
        base_image = img

    if rotation==True:
        for contour in contours:    
        # Tính toán hình chữ nhật có diện tích nhỏ nhất bao quanh contour  
            rect = cv.minAreaRect(contour)
            box = cv.boxPoints(rect)
            box = np.int0(box) 
            # box = np.int0(box) + [[-10, -10], [10, -10], [10, 10], [-10, 10]] 

            w = int(rect[1][0])
            h = int(rect[1][1])
            if w >= min_width and h >= min_height:
                output = crop_rotation(base_image, box, w, h)
                image_name = f"{i}.jpg"
                i+=1
                cv.imwrite(os.path.join(output_dir, image_name), output)
    else: 
        for contour in contours:    
        # Tính toán hình chữ nhật có diện tích nhỏ nhất bao quanh contour
            x, y, w, h = cv.boundingRect(contour)
            new_w = w + extend_box
            new_h = h + extend_box
            new_x = x - extend_box
            new_y = y - extend_box
            if w >= min_width and h >= min_height and new_x>0 and new_y>0 and x+new_w<base_image.shape[1] and y+new_h<base_image.shape[0]:
                output = base_image[new_y:y+new_h, new_x:x+new_w]
                image_name = f"{count}.png"
                cv.imwrite(os.path.join(output_dir, image_name), output)
                count+=1

In [6]:
folder_path = '/home/dotronghiep/Documents/Research/Peanuts_Anomaly_Detection_PAD/test/color_images'
count = 0
for filename in os.listdir(folder_path):
    img_path = os.path.join(folder_path, filename)    
    # Gọi hàm a trên ảnh
    segmentation(img_path, background=False, rotation=False)